#### Streamlined code to run batch'esque 
#### differential expressions with edgeR
---
##### hpb29

Date: 2021-01-26

In [1]:
library(edgeR)
library(data.table)

Loading required package: limma



In [2]:
countsfile <- 'gene_x_cells_counts_matrix.txt.gz'

In [3]:
datadir <- '2020/SLX19841/analysis/matrices/'

In [4]:
Sys.time()
counts <- fread( paste(datadir, countsfile, sep='') )
Sys.time()

[1] "2021-01-26 18:37:00 GMT"

[1] "2021-01-26 18:37:17 GMT"

First column is genes names

In [5]:
genes <- counts$V1

Header's second element onwards is cell names 

In [6]:
cells <- colnames(counts)[2:ncol(counts)]

Excise the first column

In [7]:
counts <- counts[,2:ncol(counts)]

In [8]:
dim(counts)

[1] 22656 16402

Turn data.table into a data.frame

In [9]:
counts_df = setDF(counts)

In [10]:
length(genes)

[1] 22656

In [11]:
rownames(counts_df) <- genes

In [ ]:
# gene_mask <- rowSums(counts_df) > 0
# sum(gene_mask)

# counts_df <-counts_df[gene_mask,]
# dim(counts_df)

In [13]:
rm(counts)

In [14]:
metafile <- 'metadata.txt'

In [15]:
coldata <- read.table(paste0(datadir, metafile), 
                      sep='\t', header=T)

rownames(coldata) <- coldata[,1]
coldata[,1] <- NULL
head(coldata)

,donor,organ,cluster
,<chr>,<chr>,<int>
_01_AAACCTGCAAGCCGCT.1.1,DOD1,BM,9
_01_AAACCTGTCTGGGCCA.1.1,DOD1,BM,6
_01_AAACGGGCACATTCGA.1.1,DOD1,BM,6
_01_AAAGATGCATCTCCCA.1.1,DOD1,BM,6
_01_AAAGTAGAGACGACGT.1.1,DOD1,BM,9
_01_AAAGTAGAGCCGATTT.1.1,DOD1,BM,6


In [16]:
dim(coldata)

[1] 16402     3

---

In [17]:
dex_it_out <- function(data, metadata, targets, outbasename){
    
    cell_subset <- rownames(metadata[ metadata$cluster %in% targets, ])
    selection <- data[ , cell_subset ]
    coldata_subset <- metadata[cell_subset, ]
    
    group <- factor(c(coldata_subset$organ)) # hardcoded / Edit this per comparison batch
    
    y <- DGEList(counts=selection[, cell_subset], group=group)
    print("Computing norm factors...")
    y <- calcNormFactors(y)
    design <- model.matrix(~0+group)
    
    y <- estimateDisp(y,design)
   
    # Edit these per comparison batch
    # --------------------------------------------------------------------------------
    BM_vs_SPL <- makeContrasts( groupBM-groupSPL, levels=design) # hardcoded
    
    fit <- glmFit(y,   contrast=as.vector(BM_vs_SPL) )
    lrt <- glmLRT(fit, contrast=as.vector(BM_vs_SPL) )
    # --------------------------------------------------------------------------------
    
    group_res <- topTags(lrt, sort.by = "PValue", adjust.method="fdr", n=dim(lrt$table)[1])

    head(group_res)
    
    saveRDS(lrt, paste0(datadir, outbasename, "_LRT.rds"))

    write.table(group_res, 
            file = paste0('output/', outbasename, '_DEx_results.txt'), 
            row.names = TRUE, quote = FALSE, sep='\t')
    
    #trick to handle maximum floating point accuracy
    group_res$table$FDR[group_res$table$FDR == 0.000000e+00 ] <- 1e-323
    group_res$table$PValue[group_res$table$PValue == 0.000000e+00 ] <- 1e-323
    
    er <- group_res$table
    er$genes <- rownames(er)

    er$fcsign <- sign(er$logFC)
    er$logP=-log10(er$PValue)
    er$metric= er$logP/er$fcsign


    final<-er[,c("genes", "metric")]
    
    write.table(na.exclude( final[order(final$metric, decreasing = TRUE), ] ), 
            file = paste0('output/', outbasename, '.rnk'), 
            row.names = FALSE, col.names = FALSE, quote = FALSE, sep='\t')
    
}

e.g.

A) BM vs SPL (using DOD1,2,3,4) for: 

Cluster 2 

## Cluster 2

In [18]:
Sys.time()

dex_it_out(counts_df, coldata, 
           c('2'), 
           '20210126_COMBO10_Cluster2_BM_vs_SPL')

Sys.time()

[1] "2021-01-26 18:44:05 GMT"

[1] "Computing norm factors..."


[1] "2021-01-26 18:49:55 GMT"